# Easy OCR

In [ ]:
import tensorflow as tf
from app.services.qcap import process_qcap

In [7]:
import easyocr
reader = easyocr.Reader(['id'], gpu=False)

Using CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [8]:
results = reader.readtext("./inference/jpgs/img_00001.jpg")
results

C:\Users\Ghiffari\work-folder\01a-digital-projects\2502-01-DBS Foundation Camp\04-projects\capstone-project\qtancy-machine-learning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


[([[57, 483], [71, 483], [71, 509], [57, 509]], '1', 0.9986463600755577),
 ([[115, 475], [461, 475], [461, 517], [115, 517]],
  'SPGTHY BOLOGNASE',
  0.3772504946452746),
 ([[513, 477], [637, 477], [637, 521], [513, 521]],
  '58,OQo',
  0.5079821943176018),
 ([[55, 529], [71, 529], [71, 557], [55, 557]], '1', 0.9997164212072427),
 ([[113, 520], [256, 520], [256, 562], [113, 562]],
  'PEPPER',
  0.9985323994993548),
 ([[267, 521], [341, 521], [341, 561], [267, 561]],
  'AVs',
  0.25621664945060624),
 ([[494, 515], [641, 515], [641, 566], [494, 566]],
  '165,Ooo',
  0.4151908293778719),
 ([[137, 569], [233, 569], [233, 609], [137, 609]],
  'WELL',
  0.9982210993766785),
 ([[247, 565], [343, 565], [343, 607], [247, 607]],
  'DONE',
  0.6025109968494945),
 ([[112, 610], [384, 610], [384, 658], [112, 658]],
  'WAGYU RIBEYE',
  0.8970899244688244),
 ([[498, 606], [646, 606], [646, 654], [498, 654]],
  '195,Ooo',
  0.7764594667594529),
 ([[136, 656], [386, 656], [386, 706], [136, 706]],
  'ME

In [14]:
text_lines = [res[1] for res in results]

In [17]:
joined  = " ".join(text_lines)

In [23]:
display(joined)

'1 SPGTHY BOLOGNASE 58,OQo 1 PEPPER AVs 165,Ooo WELL DONE WAGYU RIBEYE 195,Ooo MEDIUM WELL 1 ICED LEMON TEA 22,Ooo 1 FUSION TEA LYCHE 28 , OQo 1 NUTTELA BRDWNIES 35 Ooo 503,OQ7 SUBtoTAL 52,815 PB1 25,150 SVC ChrG DUE 580,965'

In [19]:
import re

def clean_ocr_text(text):
    # Normalize spacing
    text = re.sub(r'\s+', ' ', text)
    
    # Fix common OCR misreads
    text = text.replace('OQo', '000').replace('Ooo', '000').replace(',Ooo', ',000')
    text = text.replace(', OQo', ',000').replace(' , ', ' ')
    
    # Split by items or numbers
    items = re.findall(r'(\d+\s+[\w\s]+?\s+\d{2,3},000)', text)
    
    # Extract total and other charges
    subtotal = re.search(r'SUBtoTAL\s+(\d{2,3},\d{3})', text, re.IGNORECASE)
    pb1 = re.search(r'PB1\s+(\d{2,3},\d{3})', text)
    svc = re.search(r'SVC\s+ChrG\s+DUE\s+(\d{2,3},\d{3})', text)

    return {
        'items': items,
        'subtotal': subtotal.group(1) if subtotal else None,
        'pb1': pb1.group(1) if pb1 else None,
        'total': svc.group(1) if svc else None
    }


In [21]:
!pip install spacy

  Using cached numpy-2.2.6-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
    --------------------------------------- 0.3/14.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.9 MB 1.5 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/14.9 MB 1.6 MB/s eta 0:00:09
   -- ------------------------------------- 1.0/14.9 MB 1.2 MB/s eta 0:00:12
   --- ------------------------------------ 1.3/14.9 MB 1.2 MB/s eta 0:00:12
   ---- ----------------------------------- 1.6/14.9 MB 1.3 MB/s eta 0:00:11
   ---- ----------------------------------- 1.8/14.9 MB 1.3 MB/s eta 0:00:11
   ----- ---------------------------------- 2.1/14.9 MB 1.3 MB/s eta 0:00:10
   ------ --------------------------------- 2.4/14.9 MB 1.3 MB/s eta 0:00:10
   ------- -------------------------------- 2.6/14.9 MB 1.4 MB/s eta 0:00:10
   ------- --------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.17 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.2.6 which is incompatible.
langchain-community 0.2.17 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.2.6 which is incompatible.
paddlex 3.0.0 requires numpy==1.24.4; python_version < "3.12", but you have numpy 2.2.6 which is incompatible.
tensorflow-intel 2.17.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.2.6 which is incompatible.


In [22]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

nlp = English()
ruler = EntityRuler(nlp)
patterns = [
    {"label": "FOOD_ITEM", "pattern": [{"IS_DIGIT": True}, {"IS_ALPHA": True, "OP": "+"}, {"LIKE_NUM": True}]}
]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

doc = nlp("1 SPGTHY BOLOGNASE 58,000")
for ent in doc.ents:
    print(ent.text, ent.label_)

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [ ]:
def process_easyocr(folder_path: str = None, label: str = None):
    img_path = os.path.join(folder_path, img_name)
    results = reader.readtext(img_path)
    text_lines = [res[1] for res in results]  # ambil teks dari result

    print(json.dumps(parsed, indent=2, ensure_ascii=False))
    print("-" * 60)

# Inference

In [96]:
from app.services.qcap import process_qcap

In [97]:
process_qcap("inference\jpgs\img_00000.jpg")

{'menu': [{'nm': 'Nasi Campur Bali', 'cnt': '1 x', 'price': '75,000'},
  {'nm': 'Bbk Bengil Nasi', 'cnt': '1 x', 'price': '125,000'},
  {'nm': 'MilkShake Starwb', 'cnt': '1 x', 'price': '37,000'},
  {'nm': 'Ice Lemon Tea', 'cnt': '1 x', 'price': '24,000'},
  {'nm': 'Nasi Ayam Dewata', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Free Ice Tea', 'cnt': '3 x', 'price': '0'},
  {'nm': 'Organic Green Sa', 'cnt': '1 x', 'price': '65,000'},
  {'nm': 'Ice Tea', 'cnt': '1 x', 'price': '18,000'},
  {'nm': 'Ice Orange', 'cnt': '1 x', 'price': '29,000'},
  {'nm': 'Ayam Suir Bali', 'cnt': '1 x', 'price': '85,000'},
  {'nm': 'Tahu Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tempe Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tahu Telor Asin', 'cnt': '1 x', 'price': '40,000.'},
  {'nm': 'Nasi Goreng Samb', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Bbk Panggang Sam', 'cnt': '3 x', 'price': '366,000'},
  {'nm': 'Ayam Sambal Hija', 'cnt': '1 x', 'price': '92,000'},
  {'nm': 'Hot Tea', '

In [13]:
!pip install sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 3.4 MB/s eta 0:00:01
   ------------------------------- -------- 786.4/991.5 kB 1.7 MB/s eta 0:00:01
   ------------------------------- -------- 786.4/991.5 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 1.2 MB/s eta 0:00:00


In [2]:
import numpy as np
from transformers import VisionEncoderDecoderModel, DonutProcessor
from PIL import Image
import re

processor = DonutProcessor.from_pretrained(
    "naver-clova-ix/donut-base-finetuned-cord-v2")
model = VisionEncoderDecoderModel.from_pretrained(
    "naver-clova-ix/donut-base-finetuned-cord-v2")

In [12]:
from PIL import Image

def process_qcap(path):
    image_path = path
    image = Image.open(image_path)
    
    task_prompt = "<s_cord-v2>"
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
    
    pixel_values = processor(image, return_tensors="pt").pixel_values
    
    outputs = model.generate(
        pixel_values=pixel_values,
        decoder_input_ids=decoder_input_ids,
        max_length=model.decoder.config.max_position_embeddings,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )
    
    sequence = processor.batch_decode(outputs.sequences)[0]
    sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
    sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()  # remove first task start token
    return processor.token2json(sequence)

In [57]:
qcap_out = process_qcap("inference\jpgs\img_00000.jpg")
qcap_out

{'menu': [{'nm': 'Nasi Campur Bali', 'cnt': '1 x', 'price': '75,000'},
  {'nm': 'Bbk Bengil Nasi', 'cnt': '1 x', 'price': '125,000'},
  {'nm': 'MilkShake Starwb', 'cnt': '1 x', 'price': '37,000'},
  {'nm': 'Ice Lemon Tea', 'cnt': '1 x', 'price': '24,000'},
  {'nm': 'Nasi Ayam Dewata', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Free Ice Tea', 'cnt': '3 x', 'price': '0'},
  {'nm': 'Organic Green Sa', 'cnt': '1 x', 'price': '65,000'},
  {'nm': 'Ice Tea', 'cnt': '1 x', 'price': '18,000'},
  {'nm': 'Ice Orange', 'cnt': '1 x', 'price': '29,000'},
  {'nm': 'Ayam Suir Bali', 'cnt': '1 x', 'price': '85,000'},
  {'nm': 'Tahu Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tempe Goreng', 'cnt': '2 x', 'price': '36,000'},
  {'nm': 'Tahu Telor Asin', 'cnt': '1 x', 'price': '40,000.'},
  {'nm': 'Nasi Goreng Samb', 'cnt': '1 x', 'price': '70,000'},
  {'nm': 'Bbk Panggang Sam', 'cnt': '3 x', 'price': '366,000'},
  {'nm': 'Ayam Sambal Hija', 'cnt': '1 x', 'price': '92,000'},
  {'nm': 'Hot Tea', '

In [49]:
type(qcap_out)

dict

In [ ]:
val = "1,591,600"

## Qrep Model

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
labels = np.array(["lain-lain",
            "makanan",
            "minuman",
            "kesehatan",
            "elektronik",
            "fashion",
            "atk",
            "transport",
            "hiburan"])

le = LabelEncoder()
le.fit(labels)
le.transform(labels)

array([5, 6, 7, 4, 1, 2, 0, 8, 3])

In [25]:
le.inverse_transform([0])

array(['atk'], dtype='<U10')

In [82]:
SAMPLE_TEKS = "BENSIN"

def predict_category(text):
    logits = new_model.predict(tf.constant([text]), verbose=0)
    idx = tf.argmax(logits[0]).numpy()
    label = le.inverse_transform([idx])[0]
    return label 

In [84]:
prediksi = predict_category(SAMPLE_TEKS)
prediksi

'lain-lain'

In [33]:
import joblib

joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']

In [ ]:
import joblib

le = joblib.load("label_encoder.pkl")

# Merged Pipeline

In [102]:
val = process_qrep(qcap_out)

In [101]:
import pandas as pd

def format_data(qcap_out: dict):
    # Ambil dan ubah menu jadi dataframe
    menus = qcap_out.get("menu", [])
    data = pd.DataFrame(menus) if isinstance(menus, list) else pd.DataFrame([menus])
    data.rename(columns=lambda x: x.strip().lower(), inplace=True)

    # Pastikan kolom yang dibutuhkan ada
    if 'nm' in data.columns:
        data.rename(columns={'nm': 'item'}, inplace=True)
    if 'cnt' in data.columns:
        data.rename(columns={'cnt': 'qty'}, inplace=True)
        data['qty'] = data['qty'].apply(extract_int)
    if 'price' in data.columns:
        data.rename(columns={'price': 'price'}, inplace=True)
        data['price'] = data['price'].apply(str_to_int)

    return data

def process_qrep(qcap_out: dict,  model=model, le=le, report_period="bulanan"):
    """
    Proses hasil ekstraksi Qcap menjadi format laporan lengkap untuk Qrep.

    Args:
        qcap_out (dict): Output dari modul Qcap, berisi informasi transaksi.
        model: Model klasifikasi kategori item.
        le: LabelEncoder untuk label kategori item.
        report_period (str): Periode pelaporan, default "bulanan".

    Returns:
        dict: JSON terstruktur berisi detail transaksi dan klasifikasi kategori.
    """
    data = format_data(qcap_out)
    # Ambil field utama
    merchant_name = qcap_out.get("toko", "")
    date = qcap_out.get("tanggal", "")
    total_amount = str_to_int(qcap_out.get("total", 0).get("total_price", 0))
    payment_method = qcap_out.get("pembayaran", "")

    # Klasifikasi kategori
    if 'item' in data.columns:
        try:
            # Tokenisasi dan prediksi
            data['kategory'] = data['item'].apply(predict_category)
        except Exception as e:
            print(f"Kategori error: {e}")
            data['kategory'] = ""
    else:
        data['kategory'] = ""

    # Format list item
    item_list = data[['item', 'qty', 'price', 'kategory']].to_dict(orient='records')

    # Susun output JSON akhir
    return {
        "merchant_name": merchant_name,
        "date": date,
        "total_amount": total_amount,
        "item_list": item_list,
        "payment_method": payment_method,
        "report_period": report_period
    }

In [88]:
import re

def str_to_int(s, locale="US"):
    """
    Mengubah string angka format US/ID menjadi integer.
    
    Args:
        s (str): String angka, contoh "1,591,600.00" atau "1.591.600,00"
        locale (str): "US" atau "ID", default "US"
    
    Returns:
        int: Nilai bilangan bulat, desimal akan dibuang
    """
    s = s.strip()
    
    if locale.upper() == "US":
        # Hapus koma (,) lalu pisahkan dari desimal jika ada
        s_clean = re.sub(r",", "", s).split(".")[0]
    elif locale.upper() == "ID":
        # Hapus titik (.) lalu pisahkan dari desimal jika ada
        s_clean = re.sub(r"\.", "", s).split(",")[0]
    else:
        raise ValueError("locale must be either 'US' or 'ID'")
    
    return int(s_clean)


def extract_int(s):
    """
    Ekstrak digit pertama dari string dan ubah ke int.
    
    Args:
        s (str): String input, contoh "1 x", "  3pcs", "4x500ml"
    
    Returns:
        int: Nilai bilangan bulat pertama yang ditemukan. Jika tidak ada, return 0.
    """
    match = re.search(r"\d+", s)
    return int(match.group()) if match else 0

In [103]:
output = process_qrep(qcap_out, model=model, le=le)
output

{'merchant_name': '',
 'date': '',
 'total_amount': 1591600,
 'item_list': [{'item': 'Nasi Campur Bali',
   'qty': 1,
   'price': 75000,
   'kategory': 'makanan'},
  {'item': 'Bbk Bengil Nasi',
   'qty': 1,
   'price': 125000,
   'kategory': 'makanan'},
  {'item': 'MilkShake Starwb',
   'qty': 1,
   'price': 37000,
   'kategory': 'minuman'},
  {'item': 'Ice Lemon Tea', 'qty': 1, 'price': 24000, 'kategory': 'minuman'},
  {'item': 'Nasi Ayam Dewata',
   'qty': 1,
   'price': 70000,
   'kategory': 'makanan'},
  {'item': 'Free Ice Tea', 'qty': 3, 'price': 0, 'kategory': 'minuman'},
  {'item': 'Organic Green Sa',
   'qty': 1,
   'price': 65000,
   'kategory': 'lain-lain'},
  {'item': 'Ice Tea', 'qty': 1, 'price': 18000, 'kategory': 'minuman'},
  {'item': 'Ice Orange', 'qty': 1, 'price': 29000, 'kategory': 'minuman'},
  {'item': 'Ayam Suir Bali', 'qty': 1, 'price': 85000, 'kategory': 'makanan'},
  {'item': 'Tahu Goreng', 'qty': 2, 'price': 36000, 'kategory': 'makanan'},
  {'item': 'Tempe Gor

In [52]:
res_key = qcap_out['menu'].keys()
len(res_key)

2

In [55]:
data.to_dict()

{'nm': {0: 'PKT AYAM'}, 'price': {0: '33,000'}}

In [46]:
import pandas as pd
menus = pd.DataFrame(qcap_out['menu'])

ValueError: If using all scalar values, you must pass an index

In [43]:
menus

,nm,cnt,price
0,Bintang Bremer,1,"59,000"
1,Chicken H-H,1,"190,000"
2,Ades,1,"10,000"
